BioPAL

# BioPAL First AGB Tutorial

The BIOMASS Product Algorithm Laboratory (BioPAL) hosts official tools for processing and analysing ESA\'s BIOMASS mission data.

-   Website: www.biopal.org
-   Mailing: <biopal@esa.int>

_Disclamer:
this tutorial is released under MIT license, it is experimental and may change without notice._


## Tutorial Objective

In this tutorial, following topics will be discussed:
- requirements, biopal installation and environment preparation for this notebook to work
- setup and run of the Above Ground Biomass (AGB) processor (interactive notebook section)


## Requirements

- The installation procedure described here makes use of the open-source package management system [conda](https://docs.conda.io/projects/conda/en/latest/).
- Python >=3.6


## Installation and Jupyter Notebook Execution


1.  Download and unzip the current [BioPAL distribution](https://github.com/BioPAL/BioPAL) to your local hard drive.


2.  In a *conda* command window, enter in the directory where the BioPAL has been unzipped, and type the following instruction, which creates a biopal environment ( `environment.yml` is present into the main folder of BioPAL distribution ):
	
        conda env create --file environment.yml

3.  In the same *conda* command window, type the following instruction, which activates the created biopal environment:
	
        conda activate biopal

4.  In the same *conda* command window, install Jupyter notebook 
	
        conda install -c conda-forge jupyterlab

5.  In the same *conda* command window, open this Jupyter Notebook typing the following instructions:

        cd doc/Notebooks
        jupyter notebook
        

# Setup AGB Processor


## Main Ingredients

Main ingredients needed to run this tutorial:

-   `BioPAL/biopal/conf/Configuration_File.xml`: is the BioPAL configuration file (this tutorial is focused on the AGB only)

-   `BioPAL/inputs/Input_File.xml`: input file, to be set by the user before running an instance of the processing

This Notebook is supposed to be executed from `BioPAL/biopal/doc/Notebooks/BioPAL.ipynb`.

Setup of the AGB processor includes:
-   Input File check / preparation
-   Configuration File check / preparation


## BioPAL datasets

BioPAL gives easy access to several datasets that are used for examples in the documentation and testing. These datasets are hosted on the MAAP and must be downloaded for use. Contact <biopal@esa.int> to receive access to the dataset and for more information.
Each dataset is composed by two folders:
-   `dataSet`: folder containing L1 stacks of coregistered and calibrated SLC (Single Look Complex) data
-   `auxiliary_data_pf`: folder containing auxiliary products which are related to each particular dataSet 


## Prepare Input

Open the `inputs/Input_File.xml` and focus on the following sections:

-   `L2_product`: specifies the algorithm to be run, here kept to `AGB` to enable the Above Ground Biomass processing (objective of this tutorial)
-   `output_specification` section: 
   * `output_folder`: full path of the folder where processing will save the products. Each run will generate a new sub-folder tagged with the processing time stamp (only in case of single run of the complete suite, see later section)
   * `geographic_grid_sampling`:value, in meters, of the sampling for the final processed product on geographic map, valid (east and north directions)
-   `dataset_query` section:
   * `L1C_repository`: should be put equal to the `dataSet` folder described above
   * `L1C_date`start and stop values to select L1 products, in UTC format
   * `auxiliary_products_folder`: should be put equal to the `auxiliary_data_pf` folder described above; in particular, the sub-folder ReferenceAGB, contains the reference data used to calibrate the AGB estimation inversion algorithm
   * `geographic_boundaries_polygon`: a set of 3 or more latitude-longitude points describing a polygon used to select L1 products

Note: the Input_File.xml in the repository is already filled with example values.


## Prepare Configuration

The BioPAL configuration file is `biopal/conf/Configuration_File.xml` and it is already filled with default AGB parameters to get valid estimation results; main parameters the users can modify are described hereafter, for more details refer to the complete documentation.

-   `ground_cancellation` section: 
   * `enhanced_forest_height`: forest height in meters used in the processing to determine the vertical wavenumber for which the ground cancelled data is generated (only with more than two acquisitions)  
-   `estimate_agb` section: 
   * `number_of_tests`: number of tests executed for AGB estimation following a random sampling scheme of estimation and calibration points (reference)
   * `fraction_of_roi_per_test`: percentage of estimation points randomly selected for each test   
   * `fraction_of_cal_per_test`: percentage of calibration points randomly selected for each test
   * `intermediate_ground_averaging`: ground cancelled averaging window size in meters on ground: it should be less or equal to product_resolution/2 (see below)
   * `product_resolution`: output product resolution (AGB) in meters on geographic map   
   * `distance_sampling_area`: estimation points distance in meters on geographic map
   * `estimation_valid_values_limits`: validity range of the output product in t/ha, invalid values will be masked out
   

# Run AGB Processor

There are two ways to run the AGB processor:
-   execution of the "main" in a single run
-   manual execution of each specific AGB step

In both cases, the first step is to add the `biopal` folder to the Python path.
In the following code, it is supposed to have this notebook in `BioPAL/biopal/doc/Notebooks/BioPAL.ipynb`
If this is not the case, you can manually set the `biopal_path` in the following example:

In [4]:
from pathlib import Path
import sys
import os
import warnings
warnings.filterwarnings('ignore') # warnings silenced for convenience

# get the biopal folder path, supposing that we are in the working directory "biopal/doc/Notebooks/"
notebook_working_dir = Path.cwd()
biopal_path = notebook_working_dir.parent.parent
sys.path.append( str(biopal_path) )

# Try an import to verify the python path
try:
    from biopal.__main__ import biomassL2_processor_run
    
    print('The BioPAL path "{}" has been succesfully added to the python path'.format(biopal_path))

except Exception:
    raise Exception( 'The BioPAL Notebook needs to be executed from the folder "BioPAL/doc/Notebooks" and the "biopal" environment should be enabled too. It has been executed from "{}" instead.'.format(notebook_working_dir) )

The BioPAL path "C:\ARESYS_PROJ\BioPAL" has been succesfully added to the python path


# Run AGB Processor by execution of each AGB step

The AGB processor can be executed by manual run of each AGB step (APPs): each APP computes the inputs needed by the following one, and updates the input file adding each APP specific section, and (when needed) also the configuration file.

### dataset_query APP run

The first APP to be executed, the `dataset_query`, is in charge of getting from the `L1C_repository` only the stacks matching the specified temporal and geographical region of interest. 

It takes as input the `inputs\Input_File.xml` path, prepared in the above steps of this tutorial.

The input file will be updated adding the new section `stack_based_processing` needed to the following APP.
The updated input file will be saved into the output folder.

In [5]:
input_path = str( Path.home().joinpath( biopal_path, "inputs", "Input_File.xml") ) 
from biopal.dataset_query.dataset_query import dataset_query

# Initialize dataset query APP (no configuration file needed in this case)
dataset_query_obj = dataset_query()

# Run dataset query APP
print('Query started...')
input_path_from_query, _, _  = dataset_query_obj.run( input_path )

print('The Input_File has been updated with the new section "stack_based_processing" and saved to {}'.format(input_path_from_query))

Query started...
Query completed.
The Input_File has been updated with the new section "stack_based_processing" and saved to C:\bio\outNew\AGB\InputFile_StackBasedProcessingAGB.xml


Hereafter we check the new `stack_based_processing` section added to the input file: as an example we print all the SLC SAR images acquisition IDs selected by the query and the path of the DTM projected in radar coordinates.

In [3]:
    from biopal.io.xml_io import parse_input_file

    # read the updated input file:
    input_params_obj = parse_input_file(input_path_from_query)
    
    # get the new section stack_based_processing:
    stack_based_processing_obj = input_params_obj.stack_based_processing
    
    # read the IDs of the stacks found from the query:
    found_stack_ids = stack_based_processing_obj.stack_composition.keys()
    print('SLC SAR images (slant range geometry) stacks and acquisitions found from the query')
    for stack_id, acquisition_ids in stack_based_processing_obj.stack_composition.items():
        print('\n    Stack:')
        print('    ', stack_id)
        print('        Acquisitions:')
        for acq_id in acquisition_ids:
            print('        ', acq_id)
    dataSet_path = input_params_obj.dataset_query.L1C_repository
    print('\n The above stacks with the acquisitions can be found at following path:\n {}'.format( dataSet_path ))
    
    # Print the path of the DTM projected in radar coordinates
    reference_height_path = Path.home().joinpath(
        dataSet_path,
        stack_based_processing_obj.reference_height_file_names[stack_id]
        )
    print('\n Full path of the reference height file valid for the above stack {}: \n {}'.format(stack_id, reference_height_path))                                                            

SLC SAR images (slant range geometry) stacks and acquisitions found from the query

    Stack:
     GC_02_H_230.00_RGSW_00_RGSBSW_00_AZSW_00
        Acquisitions:
         GC_02_H_230.00_RGSW_00_RGSBSW_00_AZSW_00_BSL_00
         GC_02_H_230.00_RGSW_00_RGSBSW_00_AZSW_00_BSL_01
         GC_02_H_230.00_RGSW_00_RGSBSW_00_AZSW_00_BSL_02

    Stack:
     GC_02_H_275.00_RGSW_00_RGSBSW_00_AZSW_00
        Acquisitions:
         GC_02_H_275.00_RGSW_00_RGSBSW_00_AZSW_00_BSL_00
         GC_02_H_275.00_RGSW_00_RGSBSW_00_AZSW_00_BSL_01
         GC_02_H_275.00_RGSW_00_RGSBSW_00_AZSW_00_BSL_02

 The above stacks with the acquisitions can be found at following path:
 C:\bio\demo_lope_two\dataSet

 Full path of the reference height file valid for the above stack GC_02_H_275.00_RGSW_00_RGSBSW_00_AZSW_00: 
 C:\bio\demo_lope_two\auxiliary_data_pf\Geometry\ReferenceHeight\GC_02_H_275.00_RGSW_00_RGSBSW_00_AZSW_00


*Stefanie*: with matplotlib/plotly plot here one of the SLCs and the reference height printed above:
-   10*log10( abs(SLC)^2 ) (rows: slant range, columns: azimuth, use pixel spacing for representing in meters rather than pixel index)
-   reference height in meters (rows: slant range, columns: azimuth, use pixel spacing for representing in meters rather than pixel index)

Notes: 
Read pixel spacings from one of the dataSet data i.e. /demo_lope_two\dataSet\GC_02_H_230.00_RGSW_00_RGSBSW_00_AZSW_00_BSL_00\GC_02_H_230.00_RGSW_00_RGSBSW_00_AZSW_00_BSL_00_0001.xml
<LinesStep unit="m">3.63676</LinesStep> is azimuth pixel spacing
<SamplesStep unit="m">11.988876</SamplesStep> is slant range pixel spacing

In [ ]:
# plots

### StackBasedProcessingAGB APP run

The second APP to be executed, is the `StackBasedProcessingAGB`, which takes the above stacks and preprocesses them with ground cancellation and geocoding.

It takes as input:
-   the Input File path, the one prepared by the `dataset_query` APP, containing the new `stack_based_processing` section
-   the Configuration File path

The input file will be updated in the end adding the new section `core_processing_agb`for the following APP.
The configuration file is updated too.
The updated input file and configuration file will be saved into output folder.

In [9]:
from biopal.agb.main_AGB import StackBasedProcessingAGB
configuration_file_path = str( Path.home().joinpath( biopal_path, "biopal", "conf", "Configuration_File.xml" ) )

# Initialize Stack Based Processing AGB APP
stack_based_processing_obj = StackBasedProcessingAGB( configuration_file_path )

# Run Stack Based Processing AGB APP
print('AGB stack-based processing APP started...')
input_file_from_stack_based, configuration_file_updated = stack_based_processing_obj.run( input_path_from_query )
    
    
print('The Input_File has been updated with the new section "core_processing_agb" and saved to {}'.format(input_file_from_stack_based))

AGB stack-based processing APP started...
AGB stack-based processing APP ended correctly.

The Input_File has been updated with the new section "CoreProcessingAGB" and saved to C:\bio\outNew\AGB\Input_File_CoreProcessingAGB.xml


Hereafter we print the paths of some of the computed data, all stored in the output folder.

In [10]:
# Some of the computed optuput paths:
input_params_obj = parse_input_file(input_file_from_stack_based)
output_folder = input_params_obj.output_specification.output_folder

ground_canc_sr_path = Path.home().joinpath( 
    output_folder, 
    'Products', 'breakpoints', 'ground_cancelled_SR_GC_02_H_230.00_RGSW_00_RGSBSW_00_AZSW_00.npy')
print('\n Path of ground cancelled data in slant range geometry: \n {}'.format(ground_canc_sr_path))

ground_canc_gr_path = Path.home().joinpath( 
    output_folder, 
    'Products','temp','geocoded','GC_02_H_230.00_RGSW_00_RGSBSW_00_AZSW_00','sigma0_hh.tif')
print('\n  Path of ground cancelled HH data, geocoded: \n {}'.format(ground_canc_gr_path))

theta_inc_path = Path.home().joinpath( 
    output_folder, 
    'Products','temp','geocoded','GC_02_H_230.00_RGSW_00_RGSBSW_00_AZSW_00','theta.tif')
print('\n  Path of incidence angle map, geocoded: \n {}'.format(theta_inc_path))



 Path of ground cancelled data in slant range geometry: 
 C:\bio\outNew\AGB\AGB\Products\breakpoints\ground_cancelled_SR_GC_02_H_230.00_RGSW_00_RGSBSW_00_AZSW_00.npy

  Path of ground cancelled HH data, geocoded: 
 C:\bio\outNew\AGB\AGB\Products\temp\geocoded\GC_02_H_230.00_RGSW_00_RGSBSW_00_AZSW_00\sigma0_hh.tif

  Path of incidence angle map, geocoded: 
 C:\bio\outNew\AGB\AGB\Products\temp\geocoded\GC_02_H_230.00_RGSW_00_RGSBSW_00_AZSW_00\theta.tif


*Stefanie*: plot here:
-   ground_canc_sr_path: 10*log10( abs( GC_SR_HH )^2 )(SAR coordinates Range/Azimuth) (rows: slant range, columns: azimuth, use pixel spacing for representing in meters rather than pixel index)
-   ground_canc_gr_path: 10*log10( abs( GC_GR_HH )^2 ) (rows: north, columns: east, use pixel spacing from geotif geotransform for representing in meters rather than pixel index)
-   theta_inc_path: the incidence angle in degrees (rows: north, columns: east, use pixel spacing from geotif geotransform for representing in meters rather than pixel index)

Notes: 
GC_SR_HH = numpy.load(ground_canc_sr_path, allow_pickle=True).item(0)['hh']

Read pixel spacings from one of the dataSet data i.e. /demo_lope_two\dataSet\GC_02_H_230.00_RGSW_00_RGSBSW_00_AZSW_00_BSL_00\GC_02_H_230.00_RGSW_00_RGSBSW_00_AZSW_00_BSL_00_0001.xml
<LinesStep unit="m">3.63676</LinesStep> is azimuth pixel spacing
<SamplesStep unit="m">11.988876</SamplesStep> is slant range pixel spacing


In [ ]:
# plots

### CoreProcessingAGB APP run

The third and last APP to be executed is `CoreProcessingAGB`, which is in charge of performing the AGB estimation.

It takes as input:
-   the Input File path, the one prepared by the `stack_based_processing` APP, containingh the new `core_processing_agb` section
-   the Configuration File path, the one updated by the `stack_based_processing` APP

In [ ]:
from biopal.agb.main_AGB import CoreProcessingAGB

# Initialize Core Processing AGB APP
agb_processing_obj = CoreProcessingAGB( configuration_file_updated )
    
# Run Main APP #2: AGB Core Processing
print('AGB core-processing APP started...')
agb_processing_obj.run(input_file_from_stack_based)

AGB core-processing APP started...


Hereafter we print the paths of the final product and the calibration product used in the algorithm.
Products are geocoded to [EQUI7 map](https://github.com/TUW-GEO/Equi7Grid) 

In [8]:
# Final products full paths:
input_params_obj = parse_input_file(input_file_from_stack_based)
output_folder = input_params_obj.output_specification.output_folder

final_estimation_path = Path.home().joinpath( 
    output_folder, 
    'Products','global_AGB','AF050M','E042N048T6','agb_1_est_db_backtransf_.tif')
print('\n Path of the final AGB estimation product, in EQUI7 map geometry: \n {}'.format(final_estimation_path))
    
reference_agb_folder = input_params_obj.stack_based_processing.reference_agb_folder
calibration_path = Path.home().joinpath( 
    reference_agb_folder, 'cal_05_no_errors.tif')
print('\n Path of the input calibration data used: \n {}'.format(calibration_path))
    


 Path of the final AGB estimation product, in EQUI7 geometry: 
 C:\bio\outNew\AGB\AGB\Products\global_AGB\AF050M\E042N048T6\agb_1_est_db_backtransf_.tif

 Path of the input calibration zone SAR image used: 
 C:\bio\demo_lope_two\auxiliary_data_pf\ReferenceAGB\cal_05_no_errors.tif


*Stefanie* plot here:

final_estimation_path: the final product, estimation of the AGB in t/ha, in Equi7 geometry (rows: north, columns: east, use pixel spacing from geotif geotransform for representing in meters rather than pixel index)

For validation:

-   full LIDAR AGB image which is on the FTP here /aux_for_agb_dev/lope_lidar/lidar_agb/EQUI7_AF050M/E045N048T3/lidar_AGB_AF050M_E045N048T3.tif

   calibration_path: calibration area is a subset of the full LIDAR

-    2D istogram:
final_estimation_path vs LIDAR



## Run AGB Processor, complete 

The following code will run the complete AGB processor, by calling the `biomassL2_processor_run` function, which automatically call the APPs in sequence.

It takes as input:
-   the path of the `Input_File.xml`
-   the folder containing the `Configuration_File.xml`

In [ ]:
from biopal.__main__ import biomassL2_processor_run
input_path = str( Path.home().joinpath( biopal_path, "inputs", "Input_File.xml") ) 
configuration_folder = str( Path.home().joinpath(biopal_path, "biopal", "conf") )

# run the AGB processor
biomassL2_processor_run( input_path, configuration_folder )